In [3]:
# Importing the necessary Python libraries
import os
import json
import time
import yaml
from datetime import datetime

import polars as pl

import feedparser
import mlx_whisper
import requests

with open('../keys/watercooler_rss_feed.yaml', 'r') as f:
    wc_rss_urls = yaml.safe_load(f)
    wc_rss_patreon_url = wc_rss_urls['WATERCOOLER_PATREON_FEED']
    wc_rss_public_url = wc_rss_urls['WATERCOOLER_PUBLIC_FEED']

wc_rss_patreon = feedparser.parse(wc_rss_patreon_url)
wc_rss_public = feedparser.parse(wc_rss_public_url)

In [4]:
import re

def find_one_or_two_digit_number(text):
    match = re.search(r'\b\d{2,3}\b', text)
    if match:
        return int(match.group())
    else:
        return None

# Example usage
text = "The room number is 12 and the code is 4567."
result = find_one_or_two_digit_number(text)
print(result)  # Output: 12

12


## Public Episode Parsing

In [5]:
wc_public_episodes = []
for episode in wc_rss_public['entries']:
    episode_title = episode['title']
    episode_summary = episode['summary']
    episode_link = episode['links'][1]['href']

    raw_episode_timestamp = episode['published_parsed']
    episode_epoch_time = time.mktime(raw_episode_timestamp)
    episode_timestamp = datetime.fromtimestamp(episode_epoch_time)

    episode_num = find_one_or_two_digit_number(episode_title)

    wc_public_episodes.append({
        'title': episode_title,
        'summary': episode_summary,
        'timestamp': episode_timestamp,
        'link': episode_link,
        'episode_num': episode_num
    })

In [6]:
# Convert the list of dictionaries to a Polars DataFrame
df = pl.DataFrame(wc_public_episodes)

target_timestamp = df.filter(pl.col('title') == 'JL Cauvin | Comedian').select('timestamp').item(0, 0)
df_filtered = df.filter(pl.col('timestamp') > target_timestamp)

target_timestamp = df.filter(pl.col('title') == 'JL Cauvin | Comedian').select('timestamp').item(0, 0)
df_filtered = df.filter(pl.col('timestamp') > target_timestamp)
# df_filtered.write_csv('../data/episode-metadata/raw_wc_public_episodes.csv')

## Patreon Episode Parsing

In [34]:
wc_patreon_episodes = []
wc_movie_night_episodes = []

for episode in wc_rss_patreon['entries']:

    episode_title = episode['title']

    # Check if episode is ad-free
    if 'ad free' in episode_title.lower() or 'ad-free' in episode_title.lower():
        continue

    # Extract episode number if it has # followed by digits
    episode_num = None
    if '#' in episode_title:
        match = re.search(r'#(\d{1,3})', episode_title)
        if match:
            episode_num = int(match.group(1))

    episode_summary = episode['summary']
    episode_link = episode['links'][1]['href']

    raw_episode_timestamp = episode['published_parsed']
    episode_epoch_time = time.mktime(raw_episode_timestamp)
    episode_timestamp = datetime.fromtimestamp(episode_epoch_time)


    if 'movie night' in episode_title.lower() or 'movie-night' in episode_title.lower() or 'movie club' in episode_title.lower():
        wc_movie_night_episodes.append({
            'title': episode_title,
            'summary': episode_summary,
            'timestamp': episode_timestamp,
            'link': episode_link,
            'episode_num': episode_num
        })

    else:
        wc_patreon_episodes.append({
            'title': episode_title,
            'summary': episode_summary,
            'timestamp': episode_timestamp,
            'link': episode_link,
            'episode_num': episode_num
        })

In [35]:
df_patreon = pl.DataFrame(wc_patreon_episodes)
df_movie_night = pl.DataFrame(wc_movie_night_episodes)

In [37]:
len(df_patreon)

243

In [44]:
# Check if any episode_num appears more than once in df_patreon
duplicate_episodes = df_patreon.filter(pl.col('episode_num').is_not_null()).group_by('episode_num').agg(
    pl.count().alias('count')
).filter(pl.col('count') > 1).sort('count', descending=True)

print("Episodes that appear more than once in patreon feed:")
print(duplicate_episodes)

# Get the details of any duplicated episodes in patreon feed
if len(duplicate_episodes) > 0:
    for row in duplicate_episodes.rows():
        ep_num = row[0]
        print(f"\nDetails for duplicate episode #{ep_num}:")
        # Convert episode_num to string before comparing if it's a string in the dataframe
        if isinstance(ep_num, str):
            duplicates = df_patreon.filter(pl.col('episode_num').cast(pl.Utf8) == ep_num).select(['title', 'timestamp'])
        else:
            duplicates = df_patreon.filter(pl.col('episode_num') == ep_num).select(['title', 'timestamp'])
        print(duplicates)
        
        # Update the duplicates with 'a' and 'b' suffixes
        indices = df_patreon.with_row_index().filter(pl.col('episode_num') == ep_num).get_column('index')
        for i, idx in enumerate(indices):
            suffix = chr(97 + i)  # 'a', 'b', 'c', etc.
            df_patreon = df_patreon.with_row_count().with_columns(
                pl.when(pl.col('row_nr') == idx)
                .then(pl.lit(f"{ep_num}{suffix}"))
                .otherwise(pl.col('episode_num'))
                .alias('episode_num')
            ).drop('row_nr')



print("\nAfter fixing duplicates:")
print(f"Patreon episodes with unique episode numbers: {len(df_patreon.filter(pl.col('episode_num').is_not_null()).unique('episode_num'))}")

Episodes that appear more than once in patreon feed:
shape: (0, 2)
┌─────────────┬───────┐
│ episode_num ┆ count │
│ ---         ┆ ---   │
│ str         ┆ u32   │
╞═════════════╪═══════╡
└─────────────┴───────┘

After fixing duplicates:
Patreon episodes with unique episode numbers: 243


/var/folders/58/n_wf97y515z8xjc0hm45xl300000gn/T/ipykernel_4352/968220924.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias('count')


In [ ]:
# Save dataframe with consistent quoting for the title column
df_patreon.write_csv('../data/episode-metadata/wc_patreon_episodes.csv', quote_style="non_numeric")


In [56]:


# Remove entries from df_movie_night that begin with "Patreon"
df_movie_night = df_movie_night.filter(~pl.col('title').str.starts_with("Patreon"))

df_movie_night.write_csv('../data/episode-metadata/wc_movie_night_episodes.csv', quote_style="non_numeric")